In [ ]:
import numpy as np

import scipy as sp
import scipy.stats as stats
from scipy.stats import linregress,t
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
from matplotlib import rc
plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)
from matplotlib.colors import LinearSegmentedColormap

import json
import os
import math


from ipywidgets import interact,interactive
import ipywidgets as widgets

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
times = [-15,-10,-5,2,10,20,30,40]


In [ ]:
def ReadFiles(Dir):
    Accept = []
    for d in os.listdir(Dir):
        try:
            if('Synapse_a.json' in os.listdir(Dir+d) ):
                Accept.append(d)
        except:
            pass
    print(Accept)   
    Syn_a_arr = []
    bg_l      = []
    for a in Accept:
        with open(Dir+a+'/Synapse_a.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
    Areas_arr = []
    Dist_arr  = []
    bg_arr = []
    kk = 0
    for Syn_a in Syn_a_arr:
        kk+=1
        areas = np.array([S["area"] for S in Syn_a])
        if(not Syn_a[0]["Times"]==times):
            for l in np.sort(list(set(times)-set(Syn_a[0]["Times"]))[::-1]):
                areas = np.insert(areas,times.index(l),math.nan,-1)
                
        dist_a = [S["distance"] for S in Syn_a]
        Dist_arr  = Dist_arr+dist_a
        Areas_arr.append(areas)
    Areas_arr_b = np.vstack(Areas_arr)
    Dist_arr_b = np.array(Dist_arr)#    
    Areas_arr_b = (Areas_arr_b.T/Areas_arr_b[:,:3].mean(axis=1)).T
    return Dist_arr_b,Areas_arr_b,(Areas_arr,Dist_arr)

#===================================================================================
#===================================================================================
def barplot_annotate_brackets(num1, num2, data, center, height, yerr=None, dh=.05, barh=.05, fs=20, maxasterix=1,offS=0.6):
    """ 
    Annotate barplot with p-values.

    :param num1: number of left bar to put bracket over
    :param num2: number of right bar to put bracket over
    :param data: string to write or number for generating asterixes
    :param center: centers of all bars (like plt.bar() input)
    :param height: heights of all bars (like plt.bar() input)
    :param yerr: yerrs of all bars (like plt.bar() input)
    :param dh: height offset over bar / bar + yerr in axes coordinates (0 to 1)
    :param barh: bar height in axes coordinates (0 to 1)
    :param fs: font size
    :param maxasterix: maximum number of asterixes to write (for very small p-values)
    """

    if type(data) is str:
        text = data
    else:
        # * is p < 0.05
        # ** is p < 0.005
        # *** is p < 0.0005
        # etc.
        text = ''
        p = .05

        while data < p:
            text += '*'
            p /= 10.

            if maxasterix and len(text) == maxasterix:
                break

        if(p<0.05):
            text = r'*'
        else:
            text = r'n.s.'
    lx, ly = center[num1], height[num1]
    rx, ry = center[num2], height[num2]

    if yerr:
        ly += yerr[num1]
        ry += yerr[num2]

    ax_y0, ax_y1 = plt.gca().get_ylim()
    dh *= (ax_y1 - ax_y0)
    barh *= (ax_y1 - ax_y0)

    y = max(ly, ry) + dh

    barx = [lx, lx, rx, rx]
    bary = [y, y+barh, y+barh, y]
    mid = ((lx+rx)/2, y+barh)

    plt.plot(barx, bary, c='black')

    kwargs = dict(ha='center', va='bottom')
    if fs is not None:
        kwargs['fontsize'] = fs

    plt.text(*mid, text, **kwargs)

    return max(ly, ry)

def pltDist(S):
    A=S.T
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    x  = ax[0].hist(A[:,0],bins=30,density=True)
    ax[0].axvline(np.nanmean(A[:,0]),c='g')
    ax[0].plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'r')
    means = []
    sem = []
    for i,a in enumerate(A.T[:-1]):
        x = np.histogram(a,bins=15,density=True)
        if(i==0):
            ax[1].plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'r')
        else:
            ax[1].plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'b')
        means.append(a.mean())
        sem.append(stats.sem(a)) 
    ax[2].errorbar(x = range(len(means)),y = means, yerr=sem, label='both limits (default)')
    ax[2].set_ylim(np.nanmin(A),np.nanmax(A))
    plt.show()
    print('means:',means)
    
def pltDyn(S):
    A = S
    col ='k'
    for i in range(2):
        fig,ax = plt.subplots(1,3,figsize=(15,5))
        x = ax[0].hist(A[i+1]-A[i])
        x = ax[1].scatter((A[i+2]-A[i+1]),(A[i+1]-A[i]),c=col)
        col = np.where(A[i+1]-A[i]<0,'r',np.where(A[i+1]-A[i]>0,'b','r'))
        ax[1].set_ylabel('\huge{$i$}')
        ax[1].set_xlabel('\huge{$i+\delta i$}')
        ax[1].set_xticks([])
        ax[1].set_yticks([])
        ax[1].axvline(0,c='r')
        ax[1].axhline(0,c='r')
        x = ax[2].hist(A[i+2]-A[i+1])
        plt.show()
        ss = sum(np.logical_and((A[i+2]-A[i+1])<0,(A[i+1]-A[i])<0))
        gg = sum(np.logical_and((A[i+2]-A[i+1])>0,(A[i+1]-A[i])>0))
        gs = sum(np.logical_and((A[i+2]-A[i+1])>0,(A[i+1]-A[i])<0))
        sg = sum(np.logical_and((A[i+2]-A[i+1])<0,(A[i+1]-A[i])>0))
        g = sum((A[i+1]-A[i])>0)
        s = sum((A[i+1]-A[i])<0)
        print("Correlation: {:0.2}".format(np.corrcoef(A[i+2]-A[i+1],A[i+1]-A[i+0])[1,0]))
        print("shrink-shrink: {:2.2%}".format(ss/len(A[i+2]-A[i+1])))
        print("grow-grow: {:2.2%}".format(gg/len(A[i+2]-A[i+1])))
        print("grow-shrink: {:2.2%}".format(gs/len(A[i+2]-A[i+1])))
        print("shrink-grow: {:2.2%}".format(sg/len(A[i+2]-A[i+1])))
        print("shrink given that you grew: {:2.2%}".format(sg/g))
        print("grow given that you shrunk: {:2.2%}".format(gs/s))
        print('='*100)

In [ ]:
BaseDir = 'Data/'
Dir = BaseDir+'CA1_15Spine/Sham/'
D15,A1,(A1raw,_) = ReadFiles(Dir)
A1raw15 = np.vstack(A1raw)
Dir = BaseDir+'CA1_7Spine/Sham/'
D7,A1,(A1raw,_) = ReadFiles(Dir)
A1raw7 = np.vstack(A1raw)
Dir = BaseDir+'CA1_3Spine/Sham/'
D3,A1,(A1raw,_) = ReadFiles(Dir)
A1raw3 = np.vstack(A1raw)
Dir = BaseDir+'CA1_1Spine/Sham/'
D1,A1,(A1raw,_) = ReadFiles(Dir)
A1raw1 = np.vstack(A1raw)
Araw = np.vstack([A1raw1,A1raw3,A1raw7,A1raw15]).T
Araw = Araw[:,~np.isnan(Araw.sum(axis=0))]

# Figures

## Figure S1

### a

In [ ]:
    
def Walls(lWall,rWall):
    S = np.zeros([8,Araw.shape[-1]])
    S[0] = Araw[0]
    #S[0] = 0.1*np.ones_like(Araw[0])
    #S[0]  = np.linspace(0,1.5,830)
    for i in range(1,8):
        yi = np.random.normal(-0.000744,0.0738,len(Araw[0]))
        S[i] = S[i-1] + yi
        if((S[i]<lWall).any()):
            S[i][S[i]<lWall] -= yi[S[i]<lWall]
        if((S[i]>rWall).any()):
            S[i][S[i]>rWall] -= yi[S[i]>rWall]
    return S

S_MC = []
for k in range(1000):
    S_MC.append(Walls(np.percentile(Araw[0].flatten(),5),np.max(Araw[0])))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
#sns.kdeplot(Araw[0,:],fill=True)#ls='-',c='r',lw=3,label='Model prediction')
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Exp. data')

#plt.plot(S2[0], yhat, 'r-',lw=3,label='Experimental data')
plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    #x = np.histogram(s,bins=15,density=True,range=(np.nanmin(s), np.nanmax(s)))
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
        #plt.plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'k',alpha=0.5,
                #label='Ex. simulation')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
#sns.kdeplot(np.hstack(S_MC[:,-1,:]),ls='-',c='b',lw=3,label='Model prediction')
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model pred.')

leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.22,1.2])
plt.ylabel('')
plt.axvline(0.35,c='k',ls='--')
plt.axvline(1.2,c='k',ls='--')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### b

In [ ]:
A=Araw[~np.isnan(Araw.sum(axis=1)),:]
means_expt = []
sem_expt = []
for i,a in enumerate(A):
    means_expt.append(a.mean())
    sem_expt.append(stats.sem(a))
plt.plot(np.mean(S_MC.mean(axis=-1),axis=0))
plt.errorbar(x = np.arange(8),y = np.mean(S_MC.mean(axis=-1),axis=0), 
             yerr=sp.stats.sem(S_MC[0],axis=1) ,lw=4,c='b',label='Model mean')
h = barplot_annotate_brackets(-3,-1,stats.ttest_ind(S_MC.mean(axis=-1)[:,-3],(S_MC.mean(axis=-1)[:,-1])).pvalue,
                               np.arange(8), np.mean(S_MC.mean(axis=-1),axis=0))
h = barplot_annotate_brackets(0,3,stats.ttest_ind(S_MC.mean(axis=-1)[:,0],(S_MC.mean(axis=-1)[:,3])).pvalue,
                               np.arange(8), np.mean(S_MC.mean(axis=-1),axis=0))
h = barplot_annotate_brackets(-3,3,stats.ttest_ind(S_MC.mean(axis=-1)[:,-3],(S_MC.mean(axis=-1)[:,3])).pvalue,
                               np.arange(8), np.mean(S_MC.mean(axis=-1),axis=0))
plt.errorbar(x = np.arange(8),y = means_expt, yerr=sem_expt,lw=4,c='r',label='Expt. mean')
plt.ylim([0.4,0.6])
plt.yticks(fontsize=28)
plt.xticks(fontsize=28)
plt.yticks(rotation=90,fontsize=24)
plt.xticks(fontsize=24)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
leg = plt.legend(fontsize=24,loc=3)
leg.get_frame().set_linewidth(0.0)

### c

In [ ]:
A1 = np.roll(S_MC,-1,axis=-2)[:,:-2,:]-S_MC[:,:-2,:]
A2 = np.roll(S_MC,-2,axis=-2)[:,:-2,:]-np.roll(S_MC,-1,axis=-2)[:,:-2,:]
A1 = np.swapaxes(A1,-2,-1)
A2 = np.swapaxes(A2,-2,-1)
Corr = []
for a1,a2 in zip(A1,A2):
    for i in range(6):
        Corr.append(np.corrcoef(a1[:,i],a2[:,i])[1,0])

a1 = A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
a2 = A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
plt.scatter(A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],
            A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],c='b')
slope, intercept, r_value, p_value, std_err = stats.linregress(a1,a2)
amin = -0.2
amax = 0.26
plt.plot(np.arange(amin,amax,0.01),np.arange(amin,amax,0.01)*slope+intercept,'k',lw=3)

#x = np.corrcoef(A1,A2)
plt.ylim([-0.3,0.3])
plt.xlim([-0.3,0.3])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.text(x=-0.1,y=0.28,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)







### d

In [ ]:
def OU(theta):
    S = np.zeros_like(Araw)
    S[0] = Araw[0]#[np.clip(Araw[0],lWall,rWall)]
    for i in range(1,8):
        yi = np.random.normal(-0.000744,0.0738,len(Araw[0]))
        nS = yi
        if(i>1):
            nS += -theta*(S[i-1]-S[i-2])
        S[i] = S[i-1] + nS
    return S
S_MC = []
for k in range(1000):
    S_MC.append(OU(0.4))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=25,density=True,range=(-1, np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
#sns.kdeplot(Araw[0,:],fill=True)#ls='-',c='r',lw=3,label='Model prediction')
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Exp. data')

#plt.plot(S2[0], yhat, 'r-',lw=3,label='Experimental data')
plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    #x = np.histogram(s,bins=15,density=True,range=(np.nanmin(s), np.nanmax(s)))
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
        #plt.plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'k',alpha=0.5,
                #label='Ex. simulation')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
#sns.kdeplot(np.hstack(S_MC[:,-1,:]),ls='-',c='b',lw=3,label='Model prediction')
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model pred.')

leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0,1.8])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

In [ ]:
plt.plot(np.mean(S_MC.mean(axis=-1),axis=0))
plt.errorbar(x = np.arange(8),y = np.mean(S_MC.mean(axis=-1),axis=0), 
             yerr=sp.stats.sem(S_MC[0],axis=1) ,lw=4,c='b')
plt.ylim([0.4,0.6])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### e

In [ ]:
A1 = np.roll(S_MC,-1,axis=-2)[:,:-2,:]-S_MC[:,:-2,:]
A2 = np.roll(S_MC,-2,axis=-2)[:,:-2,:]-np.roll(S_MC,-1,axis=-2)[:,:-2,:]
A1 = np.swapaxes(A1,-2,-1)
A2 = np.swapaxes(A2,-2,-1)
Corr = []
for a1,a2 in zip(A1,A2):
    for i in range(6):
        Corr.append(np.corrcoef(a1[:,i],a2[:,i])[1,0])

a1 = A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
a2 = A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
plt.scatter(A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],
            A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],c='b')
slope, intercept, r_value, p_value, std_err = stats.linregress(a1,a2)
amin = -0.4
amax = 0.26
plt.plot(np.arange(amin,amax,0.01),np.arange(amin,amax,0.01)*slope+intercept,'k',lw=3)

#x = np.corrcoef(A1,A2)
plt.text(x=-0.15,y=0.25,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)
plt.xlim([-0.4,0.4])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()









## Fig S2

In [ ]:
plt.subplots
lowermean = []
lowerstd  = []
uppermean = []
upperstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,1.0,0.15):
    col ='k'
    A = Araw
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,1.0,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,1.0,0.15),lowerstd)

### a

In [ ]:
def ContLN3():
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+1.2*ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope)#e+0.4*sl.intercept)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*0.0933+0.0518)



    S = [Araw[0]]
    for i in range(1,8):
        sMean = fmean(S[-1]) + S[-1]
        sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = np.random.lognormal(lmean,lsig)
        k = yi+S[i-1]-S[-1]
        if(i>1):
            k+=0.3*(S[-2]-S[-1]) 
        S.append(k)
    return np.array(S)

S_MC = []
for k in range(1000):
    S_MC.append(ContLN3())
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)

plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Exp. data')


plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):

    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')

    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model pred.')
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

In [ ]:
plt.plot(np.mean(S_MC.mean(axis=-1),axis=0))
plt.errorbar(x = np.arange(8),y = np.mean(S_MC.mean(axis=-1),axis=0), 
            yerr=sp.stats.sem(S_MC[0],axis=1) ,lw=4,c='b')
plt.ylim([0.4,0.6])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### b

In [ ]:
A1 = np.roll(S_MC,-1,axis=-2)[:,:-2,:]-S_MC[:,:-2,:]
A2 = np.roll(S_MC,-2,axis=-2)[:,:-2,:]-np.roll(S_MC,-1,axis=-2)[:,:-2,:]
A1 = np.swapaxes(A1,-2,-1)
A2 = np.swapaxes(A2,-2,-1)
Corr = []
for a1,a2 in zip(A1,A2):
    for i in range(6):
        Corr.append(np.corrcoef(a1[:,i],a2[:,i])[1,0])

a1 = A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
a2 = A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
plt.scatter(A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],
            A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],c='b')
slope, intercept, r_value, p_value, std_err = stats.linregress(a1,a2)
amin = -0.3
amax = 0.26
plt.plot(np.arange(amin,amax,0.01),np.arange(amin,amax,0.01)*slope+intercept,'k',lw=3)

#x = np.corrcoef(A1,A2)
plt.text(x=-0.1,y=0.3,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

### c

In [ ]:
A_new = []
for Cutoff in np.arange(np.min(Araw),np.max(Araw),0.15):
    col ='k'
    A = Araw
    mask = np.logical_and(A[0]>Cutoff,A[0]<Cutoff+0.15)
    A2 = []
    for i in range(1,7):
        A2.append(A[i,mask])
    A_new.append(A2)
red = np.array([1,0,0])
blue = np.array([0,0,1])
vec = np.round(np.arange(np.min(Araw),np.max(Araw),0.15),2)+0.075
for i,_ in enumerate(A_new[:5]):
    plt.scatter(vec[i]*np.ones_like(np.hstack(A_new[i])),np.hstack(A_new[i]),
                color=red,alpha=0.5)
plt.boxplot([np.hstack(a) for a in A_new],widths=0.05,boxprops=dict(linewidth=3, color='k'),
        capprops=dict(linewidth=3,color='k'),
        whiskerprops=dict(linewidth=3,color='k'),
        flierprops=dict(linewidth=3,color='k', markeredgecolor='k'),
        medianprops=dict(linewidth=3,color='k'),showfliers=False,
        positions=vec)
plt.vlines(x = 0.175, ymin = np.min(Araw), ymax = np.max(Araw),color='k',lw=4)
plt.xlim([0.075,0.975])
plt.text(0.1,np.mean(Araw),'Size range',rotation=90,fontsize=36,
         verticalalignment='bottom')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

print(ax.get_ylim())
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

### d

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLN3())
S_MC = np.array(S_MC)
A_new = []
for Cutoff in np.arange(np.min(S_MC[0,:,:]),np.max(S_MC[0,:,:])+0.15,0.15):
    col ='k'
    A = S_MC[0,:,:]
    mask = np.logical_and(A[0]>Cutoff,A[0]<Cutoff+0.15)
    A2 = []
    for i in range(1,7):
        A2.append(A[i,mask])
    A_new.append(A2)
red = np.array([1,0,0])
blue = np.array([0,0,1])
vec = np.round(np.arange(np.min(S_MC[0,:,:]),np.max(S_MC[0,:,:])+0.15,0.15),3)+0.075
for i,_ in enumerate(A_new[:5]):
    plt.scatter(vec[i]*np.ones_like(np.hstack(A_new[i])),np.hstack(A_new[i]),
                color=blue,alpha=0.5)
plt.boxplot([np.hstack(a) for a in A_new],widths=0.05,boxprops=dict(linewidth=3, color='k'),
        capprops=dict(linewidth=3,color='k'),
        whiskerprops=dict(linewidth=3,color='k'),
        flierprops=dict(linewidth=3,color='k', markeredgecolor='k'),
        medianprops=dict(linewidth=3,color='k'),showfliers=False,
        positions=vec)
plt.vlines(x = 0.175, ymin = np.min(Araw), ymax = np.max(Araw),color='k',lw=4)
plt.xlim([0.075,0.975])
plt.text(0.1,np.mean(Araw),'Size range',rotation=90,fontsize=36,
         verticalalignment='bottom')
ax = plt.gca()
plt.xticks([0.275,0.425,0.575,0.725,0.875],[0.275,0.425,0.575,0.725,0.875],fontsize=20)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


### e

In [ ]:
def ContLN2(S):
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+1.2*ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope)#e+0.4*sl.intercept)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*0.0933+0.0518)



    for i in range(1,20):
        sMean = fmean(S[-1]) + S[-1]
        sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = np.random.lognormal(lmean,lsig)
        S.append(yi+S[i-1]-S[-1])
    return np.array(S)

def ContLNOU(S,theta,theta2,mu):
    fmean = sp.interpolate.interp1d(np.arange(-10.0,10.0,0.15),np.arange(-10.0,10.0,0.15)*ml.slope+ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-10.0,10.0,0.15),np.arange(-10.0,10.0,0.15)*sl.slope+0.9*sl.intercept)

    for i in range(1,8):
        sMean = fmean(S[-1])+S[-1]
        sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = sp.stats.lognorm.rvs(lsig, loc=-S[-1], scale=np.exp(lmean), size=830)
        nS = S[-1]+(yi)-theta2*(S[-1]-mu)
        if(i>1):
            nS += theta*(S[-2]-S[-1]) 
        S.append(nS)
    return np.array(S)

### e

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLN2([np.linspace(0,1.5,830)]))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Experimental \n data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model prediction')
plt.axhline(0.8080940462024767,color='k',lw=3,label='Uniform \n initial cond.')
leg = plt.legend(fontsize=24,bbox_to_anchor =(0.3,0.3))
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


### f

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLNOU([np.linspace(0,1.5,830)],0.0,0.4,0.48))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Experimental data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model prediction')
plt.axhline(0.8080940462024767,color='k',lw=3,label='Uniform initial cond.')
#leg = plt.legend(fontsize=18)
#leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


### g

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLN2([0.3*np.ones_like(Araw[0])]))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Experimental data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model prediction')
x = np.histogram([0.3*np.ones_like(Araw[0])],bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
a1 = x[0]/(sum(x[0])*(x[1][1]-x[1][0]))
a2 = (x[1][:-1]+x[1][1:])/2
plt.plot(a2,a1,color='k',lw=3,label='Delta initial cond.')
leg = plt.legend(fontsize=24,bbox_to_anchor =(0.3,0.3))
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### h

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLNOU([0.3*np.ones_like(Araw[0])],0.0,0.4,0.48))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Experimental data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model prediction')
x = np.histogram([0.3*np.ones_like(Araw[0])],bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
a1 = x[0]/(sum(x[0])*(x[1][1]-x[1][0]))
a2 = (x[1][:-1]+x[1][1:])/2
plt.plot(a2,a1,color='k',lw=3,label='Delta initial cond.')
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

## Fig S3

### a

In [ ]:
Dir = BaseDir+'CA1_15Spine/Control/'
D15,A15,(A1raw,_) = ReadFiles(Dir)
A1raw15 = np.vstack(A1raw)
Dir = BaseDir+'CA1_7Spine/Control/'
D7,A7,(A1raw,_) = ReadFiles(Dir)
A1raw7 = np.vstack(A1raw)
Dir = BaseDir+'CA1_3Spine/Control/'
D3,A3,(A1raw,_) = ReadFiles(Dir)
A1raw3 = np.vstack(A1raw)
Dir = BaseDir+'CA1_1Spine/Control/'
D1,A1,(A1raw,_) = ReadFiles(Dir)
A1raw1 = np.vstack(A1raw)
Araw = np.vstack([A1raw1,A1raw3,A1raw7,A1raw15]).T
D    = np.hstack([D1,D3,D7,D15])
D    = D[~np.isnan(Araw.sum(axis=0))]
Araw = Araw[:,~np.isnan(Araw.sum(axis=0))]
D15    = D15[~np.isnan(A1raw15.sum(axis=1))]
A1raw15=A1raw15[~np.isnan(A1raw15.sum(axis=1)),:]

In [ ]:
A=A1raw7.T[:,D7==0]
means = []
sem   = []
var   = []
var_err = []
X = []
for i,a in enumerate(A):
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(A), np.nanmax(A)))
    means.append(a.mean())
    var.append(a.var())
    var_err.append(stats.bootstrap((a,), np.var, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    sem.append(stats.sem(a))
    X.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
Y  = (x[1][:-1]+x[1][1:])/2
for i,x in enumerate(X):
    if(i<3):
        plt.plot(Y,x,'r',alpha=0.25)
    else:
        plt.plot(Y,x,'b',alpha=0.25)

X1 = X[:3]
X2 = X[3:]
    
plt.plot(Y,np.mean(X1,axis=0),'r',label='pre-stim.',lw=3)
plt.plot(Y,np.mean(X2,axis=0),'b',label='post-stim.',lw=3)
plt.fill_between(Y,np.mean(X1,axis=0)-np.std(X1,axis=0),
                 np.mean(X1,axis=0)+np.std(X1,axis=0),color='r',alpha=0.25)
plt.fill_between(Y,np.mean(X2,axis=0)-np.std(X2,axis=0),
                 np.mean(X2,axis=0)+np.std(X2,axis=0),color='b',alpha=0.25)
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([])
plt.show()
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.gca().yaxis.tick_right()
ax = plt.gca()
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.yticks([0.45,0.5,0.55],fontsize=48)
plt.xticks(fontsize=48)
plt.show()
P1 = [x/sum(x) for x in X1]
P2 = [x/sum(x) for x in X2]
boxprops = dict(linewidth=3, color='b')
whiskerprops = dict(linewidth=3, color='b')
capprops = dict(linewidth=3, color='b')
medianprops = dict( linewidth=3, color='r')
c='r'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1],positions=[1],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['pre-stim'])
c='b'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P2],positions=[2],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['post-stim'],showfliers=False)
h = barplot_annotate_brackets(0,1,stats.ttest_ind([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]
                                        ,[sum(p[p>0]*np.log2(1/p[p>0])) for p in P2]).pvalue,
                               [1,2], [3,3],fs=48)
ax = plt.gca()
plt.gca().yaxis.tick_right()
plt.xticks(fontsize=64,rotation=-20)
plt.yticks([2.6,2.8,3.0],fontsize=48)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)

### b

In [ ]:
A=A1raw7.T[:,np.logical_and(D7!=0,np.abs(D7)<4)] 
means = []
sem   = []
var   = []
var_err = []
X = []
for i,a in enumerate(A):
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(A), np.nanmax(A)))
    means.append(a.mean())
    var.append(a.var())
    var_err.append(stats.bootstrap((a,), np.var, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    sem.append(stats.sem(a))
    X.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
Y  = (x[1][:-1]+x[1][1:])/2

for i,x in enumerate(X):
    if(i<3):
        plt.plot(Y,x,'r',alpha=0.25)
    else:
        plt.plot(Y,x,'b',alpha=0.25)

X1 = X[:3]
X2 = X[3:]
    
plt.plot(Y,np.mean(X1,axis=0),'r',label='pre-stim',lw=3)
plt.plot(Y,np.mean(X2,axis=0),'b',label='post-stim',lw=3)
plt.fill_between(Y,np.mean(X1,axis=0)-np.std(X1,axis=0),
                 np.mean(X1,axis=0)+np.std(X1,axis=0),color='r',alpha=0.25)
plt.fill_between(Y,np.mean(X2,axis=0)-np.std(X2,axis=0),
                 np.mean(X2,axis=0)+np.std(X2,axis=0),color='b',alpha=0.25)
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([])
plt.show()
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.gca().yaxis.tick_right()
ax = plt.gca()
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.yticks([0.4,0.44,0.48],fontsize=48)
plt.xticks(fontsize=48)
plt.show()
P1 = [x/sum(x) for x in X1]
P2 = [x/sum(x) for x in X2]
boxprops = dict(linewidth=3, color='b')
whiskerprops = dict(linewidth=3, color='b')
capprops = dict(linewidth=3, color='b')
medianprops = dict( linewidth=3, color='r')
c='r'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1],positions=[1],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['pre-stim'])
c='b'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P2],positions=[2],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['post-stim'],showfliers=False)
h = barplot_annotate_brackets(0,1,stats.ttest_ind([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]
                                                  ,[sum(p[p>0]*np.log2(1/p[p>0])) for p in P2]).pvalue,
                               [1,2], [3.4,3.4],fs=48)
ax = plt.gca()
plt.gca().yaxis.tick_right()
plt.xticks(fontsize=64,rotation=-20)
plt.yticks([3.2,3.4],fontsize=48)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)

### c

In [ ]:
col ='k'
A = A1raw7.T[:,D7==0]
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
Del = np.array(Del)
fig = plt.figure(figsize=(24,8))
ax = fig.add_subplot(111, projection='3d')
nbins = 50
red = np.array([1,0,0])
blue = np.array([0,0,1])
muvec = []
sem = []
for i,(z,d) in enumerate(zip([120,100,80,60, 40, 20, 0],Del)):
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2
    if(i<2):
        col=red
    elif(i==2):
        col='teal'
    else:
        col=blue
    ax.bar(xs, hist, zs=z, zdir='x', 
           color=col,alpha=0.6,width=(xs[-1]-xs[0])/50)
    mu, std = stats.norm.fit(d)
    x = np.linspace(d.min(),d.max())
    p = stats.norm.pdf(x, mu, std)
    ax.plot(x,p,zs=z, zdir='x', linewidth=2,c='k')
    x = [z]*5
    y = [mu]*5
    z2 = np.linspace(0,p.max(),5)
    ax.plot(x,y,z2,'k-',linewidth=2)
    ax.yaxis.labelpad = 10
    muvec.append(mu)
    sem.append(stats.sem(d))
ax.view_init(elev=30., azim=-50)
#ax.set_aspect(aspect=1)
ax.set_xticks([])
ax.set_zticks([])
ax.set_yticks([-0.5,0.,0.5])
#ax.set_yticks(fontsize=14)
#ax.set_box_aspect((2,1,1),zoom=1.2)
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.8, 1, 1, 1.4]))
#ax.axes.set_ylim3d(bottom=-0.4, top=0.4) 
plt.show()
fig,ax = plt.subplots(figsize=(6,2))
plt.errorbar(x = np.arange(3,7),y = muvec[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(2),y = muvec[:2], yerr=sem[:2],lw=4,c='r')
plt.errorbar(x = [2],y = muvec[2], yerr=sem[2],lw=4,c='teal')
plt.plot([1,2,3],[muvec[1],muvec[2],muvec[3]],'teal',lw=4)
plt.plot([0,1],[0.2,0.2],'k-')
plt.plot([3,6],[0.2,0.2],'k-')
barplot_annotate_brackets(0, 1, stats.f_oneway(Del[0],Del[1],Del[3],Del[4],Del[5],Del[6]).pvalue, [0.5,4.5],[0.2,.2]*2
                         ,fs=32)
barplot_annotate_brackets(0, 1, stats.f_oneway(Del[0],Del[1],Del[2],Del[3],Del[4],Del[5],Del[6]).pvalue, [2,2],[0.10]*2
                         ,fs=32)
ax.set_ylim(ax.get_ylim()[0],ax.get_ylim()[1]+0.02)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### d

In [ ]:
col ='k'
A = A1raw7.T[:,np.logical_and(D7!=0,abs(D7)<4)]
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
Del = np.array(Del)
fig = plt.figure(figsize=(24,8))
ax = fig.add_subplot(111, projection='3d')
nbins = 50
red = np.array([1,0,0])
blue = np.array([0,0,1])
muvec = []
sem = []
for i,(z,d) in enumerate(zip([120,100,80,60, 40, 20, 0],Del)):
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2
    if(i<2):
        col=red
    elif(i==2):
        col='teal'
    else:
        col=blue
    ax.bar(xs, hist, zs=z, zdir='x', 
           color=col,alpha=0.6,width=(xs[-1]-xs[0])/50)
    mu, std = stats.norm.fit(d)
    x = np.linspace(d.min(),d.max())
    p = stats.norm.pdf(x, mu, std)
    ax.plot(x,p,zs=z, zdir='x', linewidth=2,c='k')
    x = [z]*5
    y = [mu]*5
    z2 = np.linspace(0,p.max(),5)
    ax.plot(x,y,z2,'k-',linewidth=2)
    ax.yaxis.labelpad = 10
    muvec.append(mu)
    sem.append(stats.sem(d))
ax.view_init(elev=30., azim=-50)
#ax.set_aspect(aspect=1)
ax.set_xticks([])
ax.set_zticks([])
ax.set_yticks([-0.5,0.,0.5])
#ax.set_yticks(fontsize=14)
#ax.set_box_aspect((2,1,1),zoom=1.2)
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.8, 1, 1, 1.4]))
#ax.axes.set_ylim3d(bottom=-0.4, top=0.4) 
plt.show()
fig,ax = plt.subplots(figsize=(6,2))
plt.errorbar(x = np.arange(3,7),y = muvec[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(2),y = muvec[:2], yerr=sem[:2],lw=4,c='r')
plt.errorbar(x = [2],y = muvec[2], yerr=sem[2],lw=4,c='teal')
plt.plot([1,2,3],[muvec[1],muvec[2],muvec[3]],'teal',lw=4)
plt.plot([0,3],[0.09,0.09],'k-')
plt.plot([5,6],[0.09,0.09],'k-')
barplot_annotate_brackets(0, 1, 
                          stats.f_oneway(Del[0],Del[1],Del[2],Del[3],Del[5],Del[6]).pvalue, [2,5.5],[0.09]*2,
                                                  fs=32)
barplot_annotate_brackets(0, 1, stats.f_oneway(Del[0],Del[1],Del[2],Del[3],Del[4],Del[5],Del[6]).pvalue, [4,4],[0.03]*2
                                                  ,fs=32)




ax.set_ylim(ax.get_ylim()[0],ax.get_ylim()[1]+0.01)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)